# Testing Domain V2 TypedDict Contracts

This notebook demonstrates the new TypedDict-based domain architecture in the Kili Python SDK.

## Features Tested:
- TypedDict contracts with validation
- View wrappers for ergonomic access
- DataFrame adapters
- Domain helper functions

## Setup

Using the local test API:
- API_KEY: ``
- ENDPOINT: `http://localhost:4001/api/label/v2/graphql`

In [ ]:
# Import the new domain_v2 contracts and utilities
import json
from typing import List

# For testing with the API
from kili.client import Kili
from kili.domain_v2 import (
    AssetContract,
    AssetView,
    LabelView,
    ProjectView,
    UserContract,
    UserView,
    validate_asset,
    validate_label,
    validate_project,
    validate_user,
)
from kili.domain_v2.adapters import ContractValidator, DataFrameAdapter

In [ ]:
# Configure API connection
API_KEY = ""
ENDPOINT = "http://localhost:4001/api/label/v2/graphql"

# Initialize Kili client
kili = Kili(api_key=API_KEY, api_endpoint=ENDPOINT)

print("✅ Connected to Kili API")

## 1. Asset Contracts and Validation

Test the AssetContract TypedDict and validation.

In [ ]:
# Example: Create a sample asset contract
sample_asset: AssetContract = {
    "id": "asset-test-123",
    "externalId": "test-image-001",
    "content": "https://example.com/image.jpg",
    "status": "TODO",
    "labels": [],
    "createdAt": "2025-01-15T10:00:00Z",
    "updatedAt": "2025-01-15T10:00:00Z",
}

# Validate the asset contract
validated_asset = validate_asset(sample_asset)
print("✅ Asset contract validated successfully")
print(json.dumps(validated_asset, indent=2))

In [ ]:
# Create an AssetView for ergonomic access
asset_view = AssetView(validated_asset)

print(f"Display Name: {asset_view.display_name}")
print(f"Has Labels: {asset_view.has_labels}")
print(f"Is Labeled: {asset_view.is_labeled}")
print(f"Label Count: {asset_view.label_count}")

# Convert back to dict (returns reference, no copy)
asset_dict = asset_view.to_dict()
print(f"\nOriginal dict ID matches: {asset_dict['id'] == sample_asset['id']}")

## 2. Fetch Real Assets from API

Query assets from the test API and convert to TypedDict contracts.

In [ ]:
# Fetch projects to get a project ID
try:
    projects = kili.projects(first=1)
    if projects:
        project_id = projects[0]["id"]
        print(f"✅ Found project: {project_id}")
    else:
        print("⚠️  No projects found. Using sample data.")
        project_id = None
except Exception as e:
    print(f"⚠️  Could not fetch projects: {e}")
    print("Using sample data instead.")
    project_id = None

In [ ]:
# Fetch assets if we have a project
if project_id:
    try:
        assets_data = kili.assets(
            project_id=project_id,
            first=5,
            fields=[
                "id",
                "externalId",
                "content",
                "status",
                "createdAt",
                "updatedAt",
                "labels.id",
                "labels.author.email",
            ],
        )

        print(f"✅ Fetched {len(assets_data)} assets from API")

        # Validate assets as AssetContracts
        validated_assets: List[AssetContract] = []
        for asset in assets_data:
            try:
                validated = validate_asset(asset)  # type: ignore
                validated_assets.append(validated)
            except Exception as e:
                print(f"⚠️  Validation failed for asset {asset.get('id')}: {e}")

        print(f"✅ Validated {len(validated_assets)} assets")

        # Display first asset
        if validated_assets:
            first_asset = AssetView(validated_assets[0])
            print("\nFirst Asset:")
            print(f"  ID: {first_asset.id}")
            print(f"  External ID: {first_asset.external_id}")
            print(f"  Display Name: {first_asset.display_name}")
            print(f"  Status: {first_asset.status}")
            print(f"  Label Count: {first_asset.label_count}")

    except Exception as e:
        print(f"⚠️  Could not fetch assets: {e}")
        validated_assets = []
else:
    validated_assets = []

## 3. DataFrame Adapters

Test converting contracts to pandas DataFrames.

In [ ]:
# Use real assets if available, otherwise create sample data
if validated_assets:
    test_assets = validated_assets
else:
    # Create sample data for testing
    test_assets = [
        validate_asset(
            {
                "id": f"asset-{i}",
                "externalId": f"test-{i:03d}",
                "content": f"https://example.com/image-{i}.jpg",
                "status": "TODO" if i % 2 == 0 else "ONGOING",
                "labels": [],
                "createdAt": "2025-01-15T10:00:00Z",
            }
        )
        for i in range(5)
    ]

# Convert to DataFrame
adapter = DataFrameAdapter()
df = adapter.to_dataframe(test_assets)

print("✅ Converted assets to DataFrame")
print(f"\nDataFrame shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
df.head()

In [ ]:
# Convert back from DataFrame to contracts
reconstructed_assets = adapter.from_dataframe(df)

print(f"✅ Converted DataFrame back to {len(reconstructed_assets)} AssetContracts")
print("\nFirst reconstructed asset:")
print(json.dumps(reconstructed_assets[0], indent=2, default=str))

## 4. Label Contracts and Helper Functions

Test label contracts and domain helper functions.

In [ ]:
from kili.domain_v2.label import filter_labels_by_type, sort_labels_by_created_at

# Create sample labels
sample_labels = [
    validate_label(
        {
            "id": "label-1",
            "labelType": "DEFAULT",
            "createdAt": "2025-01-15T10:00:00Z",
            "author": {"email": "user1@example.com"},
        }
    ),
    validate_label(
        {
            "id": "label-2",
            "labelType": "REVIEW",
            "createdAt": "2025-01-15T09:00:00Z",
            "author": {"email": "user2@example.com"},
        }
    ),
    validate_label(
        {
            "id": "label-3",
            "labelType": "DEFAULT",
            "createdAt": "2025-01-15T11:00:00Z",
            "author": {"email": "user3@example.com"},
        }
    ),
]

print("Created 3 sample labels")

In [ ]:
# Sort labels by creation date
sorted_labels = sort_labels_by_created_at(sample_labels)

print("Labels sorted by creation date:")
for label in sorted_labels:
    view = LabelView(label)
    print(f"  {view.id}: {view.created_at} ({view.label_type})")

In [ ]:
# Filter labels by type
default_labels = filter_labels_by_type(sample_labels, "DEFAULT")

print(f"\nFiltered to DEFAULT labels: {len(default_labels)} found")
for label in default_labels:
    view = LabelView(label)
    print(f"  {view.id}: {view.label_type}")

## 5. Project and User Contracts

Test project and user contracts with real API data if available.

In [ ]:
# Fetch a project from the API
if project_id:
    try:
        project_data = kili.projects(
            project_id=project_id,
            fields=[
                "id",
                "title",
                "description",
                "inputType",
                "createdAt",
                "updatedAt",
            ],
        )

        if project_data:
            project_contract = validate_project(project_data[0])  # type: ignore
            project_view = ProjectView(project_contract)

            print("✅ Fetched and validated project:")
            print(f"  Title: {project_view.title}")
            print(f"  Description: {project_view.description or 'N/A'}")
            print(f"  Input Type: {project_view.input_type}")
            print(f"  Display Name: {project_view.display_name}")
    except Exception as e:
        print(f"⚠️  Could not fetch project: {e}")
else:
    print("⚠️  No project ID available")

In [ ]:
# Create a sample user contract
sample_user: UserContract = {
    "id": "user-123",
    "email": "test@example.com",
    "firstname": "Test",
    "lastname": "User",
    "activated": True,
}

validated_user = validate_user(sample_user)
user_view = UserView(validated_user)

print("✅ Created sample user:")
print(f"  Email: {user_view.email}")
print(f"  Full Name: {user_view.full_name}")
print(f"  Display Name: {user_view.display_name}")
print(f"  Activated: {user_view.is_activated}")

## 6. Batch Validation with ContractValidator

Test batch validation with error collection.

In [ ]:
# Create a mix of valid and invalid asset data
mixed_data = [
    {"id": "asset-1", "externalId": "test-1", "content": "url1"},  # Valid
    {"id": 123, "externalId": "test-2"},  # Invalid: id should be string
    {"id": "asset-3", "externalId": "test-3", "content": "url3"},  # Valid
    {"externalId": "test-4"},  # Invalid: missing id
]

# Batch validate with error collection
validator = ContractValidator()
results = validator.validate_batch(mixed_data, AssetContract)

print("Validation results:")
print(f"  Valid: {len(results['valid'])}")
print(f"  Invalid: {len(results['invalid'])}")
print("\nErrors:")
for error in results["errors"]:
    print(f"  Index {error['index']}: {error['error']}")

## 7. Performance Test

Test performance with a larger dataset.

In [ ]:
import time

# Create a large dataset
large_dataset = [
    {
        "id": f"asset-{i}",
        "externalId": f"test-{i:05d}",
        "content": f"https://example.com/image-{i}.jpg",
        "status": "TODO",
    }
    for i in range(1000)
]

# Time validation
start = time.time()
validated = [validate_asset(asset) for asset in large_dataset]  # type: ignore
validation_time = time.time() - start

# Time DataFrame conversion
start = time.time()
df = adapter.to_dataframe(validated)
df_time = time.time() - start

print("Performance Test (1000 assets):")
print(f"  Validation: {validation_time:.3f}s")
print(f"  DataFrame conversion: {df_time:.3f}s")
print(f"  Total: {validation_time + df_time:.3f}s")
print(f"\nDataFrame shape: {df.shape}")

## Summary

This notebook demonstrated:

1. ✅ **TypedDict Contracts**: Type-safe dictionary schemas with `total=False`
2. ✅ **Runtime Validation**: Using `typeguard.check_type` for contract validation
3. ✅ **View Wrappers**: Ergonomic property access with frozen dataclasses
4. ✅ **DataFrame Adapters**: Seamless pandas integration
5. ✅ **Domain Helpers**: Utility functions for sorting and filtering
6. ✅ **Batch Validation**: Error collection for multiple items
7. ✅ **Performance**: Fast validation and conversion (tested with 1000 items)

The new domain_v2 architecture provides:
- Type safety without runtime overhead (TypedDict)
- Ergonomic API with View wrappers
- Easy integration with pandas
- Backward compatibility with existing dict-based code